## Introduction

This Colab showcases a multi-stage ADK agent pipeline that automates the transition from initial concept to a refined short story. The workflow includes:

- Prompt Optimization: An `Enhancer` agent refines the user's input for maximum clarity.

- Parallel Generation: Two distinct writing agents—`Creative` and `Focused`—simultaneously produce unique drafts.

- Expert Review: An `Evaluator` agent performs a comparative analysis, selecting the superior version and providing a detailed rationale for its choice.

## Imports

In [102]:
%%capture
!pip install google-adk

In [103]:
import asyncio
import json
from pydantic import BaseModel, Field
import os
import pprint

from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.parallel_agent import ParallelAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.runners import InMemoryRunner, Runner
from google.genai import types

## Configurations

In [104]:
# @title

GEMINI_API_KEY = "GEMINI_API_KEY" # @param {"type":"string"}

GEMINI_MODEL = "gemini-2.5-flash" # @param {"type":"string"}
APP_NAME = "story_contest_app"  # @param {"type":"string"}
USER_ID = "user_123"  # @param {"type":"string"}
SESSION_ID = "story_session_v1"  # @param {"type":"string"}

os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'False'


### Prompts

In [105]:
ENHANCER_AGENT_PROMPT = """
You are an expert Prompt Engineer.
Your goal is to take the user's input and improve it by adding necessary clarity, slight detail, or constraints to make it a better input for a debate.
Keep the enhanced prompt concise but clear.
Output ONLY the enhanced prompt text.
"""

In [106]:
POSITIVE_AGENT_PROMPT = """
You are a debator in favor of the prompt.
Write a VERY SHORT argument supporting this prompt:

__PROMPT_STARTS__

{enhanced_prompt}

__PROMPT_ENDS__

Output ONLY the argument text.
"""

In [107]:
NEGATIVE_AGENT_PROMPT = """
You are a debator not in favor this prompt.
Write a VERY SHORT argument debating against this prompt:

__PROMPT_STARTS__

{enhanced_prompt}

__PROMPT_ENDS__

Output ONLY the argument text.
"""

In [108]:
EVALUATOR_AGENT_PROMPT = """
You are a master moderator. Evaluate these two arguments based on the user's original prompt:

__PROMPT_STARTS__
{enhanced_prompt}
__PROMPT_ENDS__

__ARGUMENT_1_STARTS__
{positive_argument}
__ARGUMENT_1_ENDS__

__ARGUMENT_2_STARTS__
{negative_argument}
__ARGUMENT_2_ENDS__

Task: Decide which argument is more convincing.
"""

## Data Models

In [109]:
# --- 1. Define Output Schema for the Evaluator ---
class EvaluationResult(BaseModel):
    winner: str = Field(description="The name of the winning agent (e.g., 'positive' or 'negative').")
    rationale: str = Field(description="A brief explanation of why this argument was chosen.")
    best_argument_text: str = Field(description="The complete text of the winning argument.")

## Agents

The following diagram illustrates the agents within this Colab and their interaction patterns.

graphviz.svg

In [110]:
# Prompt Enhancer Agent
enhancer_agent = LlmAgent(
    name="Enhancer",
    model=GEMINI_MODEL,
    instruction=ENHANCER_AGENT_PROMPT,
    # It will automatically use the incoming message as input.
    # It will write its response to this key for the next agents to use.
    output_key="enhanced_prompt"
)

In [111]:
# positive Agent
positive_writer = LlmAgent(
    name="PositiveWriter",
    model=GEMINI_MODEL,
    instruction=POSITIVE_AGENT_PROMPT,
    generate_content_config=types.GenerateContentConfig,
    output_key="positive_argument"
)

In [112]:
# Negative Agent
negative_writer = LlmAgent(
    name="NegativeWriter",
    model=GEMINI_MODEL,
    instruction=NEGATIVE_AGENT_PROMPT,
    generate_content_config=types.GenerateContentConfig,
    output_key="negative_argument"
)

In [113]:
# Parallel Agent
parallel_writers = ParallelAgent(
    name="ParallelWriters",
    sub_agents=[positive_writer, negative_writer],
    description="Runs two different argument personas in parallel."
)

In [114]:
# Evaluator Agent
evaluator_agent = LlmAgent(
    name="Evaluator",
    model=GEMINI_MODEL,
    instruction=EVALUATOR_AGENT_PROMPT,
    output_schema=EvaluationResult,
    output_key="final_evaluation"
)

In [115]:
# Root, Sequential Agent
root_agent = SequentialAgent(
    name="ModeratorAgent",
    sub_agents=[enhancer_agent, parallel_writers, evaluator_agent],
    description="Generates two arguments in parallel and then evaluates them."
)

## Running

### Helper Functions

In [116]:
async def create_runner_session(app_name: str, user_id: str, session_id: str) -> Runner:
  runner = InMemoryRunner(agent=root_agent, app_name=app_name)

  await runner.session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id,
  )
  return runner

In [117]:
async def run_story_pipeline(runner: Runner, prompt: str):
    print(f"--- Starting argument Pipeline for: '{prompt}' ---\n")
    runner = InMemoryRunner(agent=root_agent, app_name=APP_NAME)

    await runner.session_service.create_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
    )

    message = types.Content(role='user', parts=[types.Part(text=prompt)])

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=message):

        author = event.author or "System"
        if event.is_final_response() and event.content and event.content.parts:
             print(f"[{author}] completed step.")

    session = await runner.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    eval_data = session.state.get("final_evaluation")

    if eval_data:
        try:
            # Use model_validate because eval_data is likely already a dict
            if isinstance(eval_data, dict):
                 result = EvaluationResult.model_validate(eval_data)
            else:
                 # Fallback just in case it is returned as a string sometimes
                 result = EvaluationResult.model_validate_json(eval_data)

            print("\n=== 🏆 FINAL VERDICT ===")
            pprint.pprint(f"WINNER: {result.winner}")
            pprint.pprint(f"RATIONALE: {result.rationale}")
            pprint.pprint(f"WINNING ARGUMENT:\n\"{result.best_argument_text}\"")
        except Exception as e:
            print(f"\nError parsing final data: {e}\nRaw output type: {type(eval_data)}\nRaw output: {eval_data}")
    else:
        print("\nError: No evaluation found in state.")

### Running the Agent

In [118]:
runner = await create_runner_session(APP_NAME, USER_ID, SESSION_ID)

In [119]:
# @title  {"run":"auto","vertical-output":true}
PROMPT = "8 hours of sleep is sufficient for a high school student"  # @param {"type":"string"}

In [120]:
await run_story_pipeline(runner, PROMPT)

--- Starting argument Pipeline for: '8 hours of sleep is sufficient for a high school student' ---

[Enhancer] completed step.
[PositiveWriter] completed step.
[NegativeWriter] completed step.
[Evaluator] completed step.

=== 🏆 FINAL VERDICT ===
'WINNER: NegativeWriter'
("RATIONALE: The NegativeWriter's argument is more convincing because it "
 'introduces the critical concept of individual variability in sleep needs, '
 "directly challenging the 'universally sufficient' aspect of the prompt. "
 'While 8 hours is a common guideline, asserting its universality for all high '
 "school students' optimal performance and well-being is often not supported "
 'by reality, as many may require more. This nuanced perspective makes it more '
 'realistic and compelling.')
('WINNING ARGUMENT:\n'
 '"Optimal sleep is not universal; many high school students require more than '
 '8 hours for peak performance and true well-being."')
